In [344]:
from autogluon.timeseries import TimeSeriesPredictor
import os
import torch
from torchinfo import summary
import pickle
import gluonts.torch.model.patch_tst

In [336]:
wd = os.getcwd()

In [337]:
model_predictor_pth = os.path.join(wd, "ag-20250317_170713")

In [338]:

predictor = TimeSeriesPredictor.load(model_predictor_pth)
print(type(predictor))  # List of trained models

<class 'autogluon.timeseries.predictor.TimeSeriesPredictor'>


In [339]:
model_name = "TemporalFusionTransformer"  # or "NeuralNetTorch"
model = predictor._trainer.load_model(model_name)
print(dir(model))

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_ag_params', '_apply_conformalization', '_apply_temperature_scaling', '_calculate_total_resources', '_check_fit_params', '_class_tags', '_compile', '_compiler', '_compute_fit_metadata', '_compute_permutation_importance', '_convert_proba_to_unified_form', '_covariate_regressor_fit_time_fraction', '_create_covariate_regressor', '_create_covariate_scaler', '_create_target_scaler', '_default_compiler', '_estimate_memory_usage', '_estimate_memory_usage_static', '_feature_metadata', '_features', '_features_internal', '_fit', '_fit_metadata', '_get_ag_params', '_get_child_aux_val', '_get_class_tags', '_get_compiler', '

In [340]:
nn_model = predictor._trainer.load_model("TemporalFusionTransformer").model_base
print((nn_model.gts_predictor))
print(nn_model.get_trained_params())
# input_size = (1, 100, 1)

# summary(nn_model, input_size=input_size)

None
{'max_epochs': 2}


In [341]:
print([attr for attr in dir(nn_model) if not attr.startswith("__")])


['_ag_params', '_apply_conformalization', '_apply_temperature_scaling', '_calculate_total_resources', '_check_fit_params', '_class_tags', '_compile', '_compiler', '_compute_fit_metadata', '_compute_permutation_importance', '_convert_proba_to_unified_form', '_covariate_regressor_fit_time_fraction', '_create_covariate_regressor', '_create_covariate_scaler', '_create_target_scaler', '_default_compiler', '_deferred_init_params_aux', '_dummy_gluonts_freq', '_estimate_memory_usage', '_estimate_memory_usage_static', '_feature_metadata', '_features', '_features_internal', '_fit', '_fit_metadata', '_get_ag_params', '_get_callbacks', '_get_child_aux_val', '_get_class_tags', '_get_compiler', '_get_default_ag_args', '_get_default_ag_args_ensemble', '_get_default_auxiliary_params', '_get_default_hpo_executor', '_get_default_params', '_get_default_resources', '_get_default_searchspace', '_get_default_stopping_metric', '_get_estimator', '_get_estimator_class', '_get_estimator_init_args', '_get_hpo_ba

In [342]:
models = os.path.join(os.path.join(wd, "ag-20250317_170713"), "models")
TFT = os.path.join(models, "TemporalFusionTransformer")

mod_pik = os.path.join(models, "trainer.pkl")
print(mod_pik)

/Users/jakegwinn/Documents/umich/Y5S2/urps/urps_2025/notebooks/jake_notebooks/flops/AutogluonModels/ag-20250317_170713/models/trainer.pkl


In [ ]:
# from autogluon.timeseries.models.gluonts.torch.tft import TemporalFusionTransformerModel
# import pickle

# model_path = os.path.join(TFT, "model.pkl")  # Adjust based on actual path

# # Load with pickle (since AutoGluon likely saved it this way)
# with open(model_path, "rb") as f:
#     autogluon_model = pickle.load(f)

# print(type(autogluon_model))  # Check if it's the TFT model


ModuleNotFoundError: No module named 'autogluon.timeseries.models.gluonts.torch.tft'

In [385]:

model = gluonts.torch.model.patch_tst.PatchTSTModel(context_length=96, prediction_length=1, patch_len=16, stride=8,d_model=32, nhead=4, num_encoder_layers=2, scaling="mean",activation="relu", dim_feedforward=32, dropout=0.1, norm_first=False, padding_patch="end", num_feat_dynamic_real=0)

In [386]:
from fvcore.nn import FlopCountAnalysis

# flops = FlopCountAnalysis(model, torch.randn(1, 100, 1))
# print(flops.total())


In [387]:
import torch.nn as nn

class WrappedForecastModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        batch_size, seq_len, feat_dim = x.shape

        # Dummy values for required model inputs
        return self.model(
            past_target=x.squeeze(-1),
            past_observed_values=torch.ones_like(x.squeeze(-1)),  # Typically a mask of same shape
        )

In [388]:
wrapped_model = WrappedForecastModel(model)
input_tensor = torch.randn(1, 100, 1)  # shape = (batch_size, time_steps, num_features)

from fvcore.nn import FlopCountAnalysis
flops = FlopCountAnalysis(wrapped_model, input_tensor)
print(f"Total FLOPs: {flops.total():,}")

Unsupported operator aten::ones_like encountered 1 time(s)
Unsupported operator aten::mul encountered 9 time(s)
Unsupported operator aten::abs encountered 2 time(s)
Unsupported operator aten::sum encountered 4 time(s)
Unsupported operator aten::div encountered 5 time(s)
Unsupported operator aten::where encountered 1 time(s)
Unsupported operator aten::pad encountered 1 time(s)
Unsupported operator aten::unfold encountered 1 time(s)
Unsupported operator aten::log1p encountered 1 time(s)
Unsupported operator aten::log encountered 1 time(s)
Unsupported operator aten::add encountered 7 time(s)
Unsupported operator aten::embedding encountered 1 time(s)
Unsupported operator aten::unflatten encountered 2 time(s)
Unsupported operator aten::scaled_dot_product_attention encountered 2 time(s)
Unsupported operator aten::softplus encountered 2 time(s)
Unsupported operator aten::clamp_min encountered 1 time(s)
The following submodules of the model were never called during the trace of the graph. They

Total FLOPs: 176,352


In [389]:
from ptflops import get_model_complexity_info

with torch.no_grad():
    macs, params = get_model_complexity_info(wrapped_model, (100, 1), as_strings=True)
    print(f"MACs: {macs}, Parameters: {params}")

WrappedForecastModel(
  26.02 k, 100.000% Params, 194.05 KMac, 100.000% MACs, 
  (model): gluonts.torch.model.patch_tst.module.PatchTSTModel(activation='relu', context_length=96, d_model=32, dim_feedforward=32, distr_output=gluonts.torch.distributions.studentT.StudentTOutput(beta=0.0), dropout=0.1, nhead=4, norm_first=False, num_encoder_layers=2, num_feat_dynamic_real=0, padding_patch='end', patch_len=16, prediction_length=1, scaling='mean', stride=8)
)
MACs: 194.05 KMac, Parameters: 26.02 k


In [390]:
model_small = gluonts.torch.model.patch_tst.PatchTSTModel(context_length=8, prediction_length=100, patch_len=1, stride=4,d_model=2, nhead=2, num_encoder_layers=2, scaling="mean",activation="relu", dim_feedforward=32, dropout=0.1, norm_first=False, padding_patch="end", num_feat_dynamic_real=0)

In [406]:
wrapped_model = WrappedForecastModel(model_small)
input_tensor = torch.randn(1, 100, 1)  # shape = (batch_size, time_steps, num_features)

with torch.no_grad():
    macs, params = get_model_complexity_info(wrapped_model, (100, 100), as_strings=True)
    print(f"MACs: {macs}, Parameters: {params}")

Flops estimation was not finished successfully because of the following exception:
<class 'RuntimeError'> : Sizes of tensors must match except in dimension 3. Expected size 104 but got size 100 for tensor number 1 in the list.
MACs: None, Parameters: None


Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/ptflops/pytorch_engine.py", line 68, in get_flops_pytorch
    _ = flops_model(batch)
        ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1844, in _call_impl
    return inner()
           ^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1790, in inner
    result = forward_call(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/w_/tv9kwn955c59q5gycbr00lzh0000gn/T/ipykernel_59817/1113042157.py", line 12, in forward
    return self.model(
           ^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl